## _*Using Qiskit Aqua for partition problems*_

This Qiskit Aqua Optimization notebook demonstrates how to use the VQE algorithm to compute a balanced partition of a set of numbers. This problem is known as "number partitioning" or simply "partition" in computer science.

The problem is defined as follows. We are given a set of numbers $S$, and we want to determine a partition of $S$ into disjoint sets $S_1, S_2$ such that $|\sum_{a \in S_1} - \sum_{a \in S_2}|$ is minimized.

The list of numbers provided as an input is used first to generate an Ising Hamiltonian, which is then passed as an input to VQE.  As a reference, this notebook also computes the maximum stable set using the Exact Eigensolver classical algorithm and the solver embedded in the commercial IBM CPLEX product (if it is available in the system and the user has followed the necessary configuration steps in order for Qiskit Aqua to find it).  Please refer to the Qiskit Aqua Optimization documentation for installation and configuration details for CPLEX.

In [1]:
import numpy as np
from qiskit import BasicAer
from qiskit.aqua import QuantumInstance
from qiskit.optimization.ising import partition
from qiskit.aqua.algorithms import ExactEigensolver, VQE
from qiskit.aqua.components.optimizers import L_BFGS_B
from qiskit.aqua.components.variational_forms import RYRZ
from qiskit.optimization.ising.common import read_numbers_from_file, random_number_list, sample_most_likely

Here an Operator instance is created for our Hamiltonian. In this case the Paulis are from an Ising Hamiltonian of the number partitioning problem (expressed in minimization form). Rather than minimizing the absolute value of the difference of the sum of numbers into two sets, we minimize the difference square.

We load a small instance of the problem.

In [2]:
number_list = read_numbers_from_file('sample.partition')
qubitOp, offset = partition.get_operator(number_list)
print(number_list)

[ 1  3  4  7 10 13 15 16]


We also offer a function to generate a set of numbers as a input.

In [3]:
if True:
    np.random.seed(8123179)
    number_list = random_number_list(5, weight_range=25)
    qubitOp, offset = partition.get_operator(number_list)
    print(number_list)

[ 9  8 23  4  2]


Here we test for the presence of algorithms we want to use in this notebook. If Aqua is installed correctly `ExactEigensolver` and `VQE` will always be found. `CPLEX.Ising` is dependent on IBM CPLEX being installed (see introduction above). CPLEX is *not required* but if installed then this notebook will demonstrate the `CPLEX.Ising` algorithm , that uses CPLEX, to compute partition as well.

In [4]:
to_be_tested_algos = ['ExactEigensolver', 'CPLEX.Ising', 'VQE']
print(to_be_tested_algos)

['ExactEigensolver', 'CPLEX.Ising', 'VQE']


We can now use the Operator without regard to how it was created. Here we will use the ExactEigensolver first to return the smallest eigenvalue. Backend is not required since this is computed classically not using quantum computation. The result is a dictionary.

In [5]:
result = ExactEigensolver(qubitOp).run()

x = sample_most_likely(result['eigvecs'][0])
print('energy:', result['energy'])
print('partition objective:', result['energy'] + offset)
print('solution:', x)
print('solution objective:', partition.partition_value(x, number_list))

energy: -694.0
partition objective: 0.0
solution: [1. 1. 0. 1. 1.]
solution objective: 0


*Note*: IBM CPLEX is an _optional_ installation addition for Aqua. If installed then the Aqua CPLEX.Ising algorithm will be able to be used. If not, then solving this problem using this particular algorithm will simply be skipped. 

We change the configuration parameters to solve it with the CPLEX backend. The CPLEX backend can deal with a particular type of Hamiltonian called Ising Hamiltonian, which consists of only Pauli Z at most second order and often for combinatorial optimization problems that can be formulated as quadratic unconstrained binary optimization problems, such as the Number Partitioning problem.

Note that for a Number Partitioning problem, since we are computing a bipartition of the set $S$, every binary vector $x$ and its complement (i.e., the vector $y$ such that $y_j = 1 - x_j$ for all $j$) represent exactly the same solution, and will have the same objective function value. Different solution methods may return solutions that look different, but in fact have the same objective function value.

In [6]:
try:
    from qiskit.aqua.algorithms import CPLEX_Ising
    result = CPLEX_Ising(qubitOp, display=0).run()
   
    x_dict = result['x_sol']
    print('energy:', result['energy'])
    print('time:', result['eval_time'])
    print('partition objective:', result['energy'] + offset)
    x = np.array([x_dict[i] for i in sorted(x_dict.keys())])
    print('solution:', x)
    print('solution objective:', partition.partition_value(x, number_list))
except Exception as ex:
    print(str(ex))


CPLEX is not installed. See https://www.ibm.com/support/knowledgecenter/SSSA5P_12.8.0/ilog.odms.studio.help/Optimization_Studio/topics/COS_home.html


Now we want VQE and so change it and add its other configuration parameters. VQE also needs and optimizer and variational form. While we can omit them from the dictionary, such that defaults are used, here we specify them explicitly so we can set their parameters as we desire.

In [7]:
vqe = VQE(qubitOp,
          RYRZ(qubitOp.num_qubits, depth=3, entanglement='linear'),
          L_BFGS_B(maxfun=6000))
result = vqe.run(QuantumInstance(BasicAer.get_backend('statevector_simulator')))

x = sample_most_likely(result['eigvecs'][0])
print('energy:', result['energy'])
print('time:', result['eval_time'])
print('partition objective:', result['energy'] + offset)
print('solution:', x)
print('solution objective:', partition.partition_value(x, number_list))

energy: -693.99999999869
time: 60.2443962097168
partition objective: 1.3100134310661815e-09
solution: [1. 1. 0. 1. 1.]
solution objective: 0
